# LHS is not available on TS machines and will be performed elsewhere

# calculate the stress state of the reference block

### For the JD_Sula_2005_gmc grid, there are 10 k layers. Reservoir starts at k=6. Use block (50, 1, 6) as reference block for *STRESSGRAD calculation. Its grid top = 670.7188 m and bottom = 671.9521 m.

In [1]:
grid_top = 670.7188
grid_bottom = 671.9521
grid_ave = (grid_top + grid_bottom)/2
print(grid_ave)

671.33545


In [3]:
# This code has been verified using the StressTransform3D function
import numpy as np
SH=25
Sh=14.6
beta = 300-90 #rotate from SH to x axis
cos_2beta = np.cos(np.radians(2*beta))
sin_2beta = np.sin(np.radians(2*beta))
sigma_x_grad=(SH+Sh)/2 + (SH-Sh)/2*cos_2beta
sigma_y_grad=(SH+Sh)/2 - (SH-Sh)/2*cos_2beta
tau_xy_grad = (SH-Sh)/2*sin_2beta
print('sigma_x gradient =',sigma_x_grad)
print('sigma_y gradient =',sigma_y_grad)
print('tau_xy gradient =',tau_xy_grad)

sigma_x gradient = 22.4
sigma_y gradient = 17.200000000000003
tau_xy gradient = 4.503332099679082


In [7]:
print('For the reference block, the effective stresses are:')
print('sigma_x = ', (sigma_x_grad - 10) * grid_ave, ' kPa')
print('sigma_y = ', (sigma_y_grad - 10) * grid_ave, ' kPa')
print('sigma_z = ', (22.7 - 10) * grid_ave, ' kPa')
print('tau_xy = ', tau_xy_grad * grid_ave, ' kPa')

For the reference block, the effective stresses are:
sigma_x =  8324.55958  kPa
sigma_y =  4833.615240000002  kPa
sigma_z =  8525.960215  kPa
tau_xy =  3023.246481637502  kPa


### check one point: pretty close

In [20]:
grid_top2 = 685.8252
grid_bottom2 = 686.126
grid_ave2 = (grid_top2 + grid_bottom2)/2

print('For the checking block, the effective stresses are:')
print('sigma_x = ', (sigma_x_grad - 10) * grid_ave2, ' kPa')
print('sigma_y = ', (sigma_y_grad - 10) * grid_ave2, ' kPa')
print('sigma_z = ', (22.7 - 10) * grid_ave2, ' kPa')
print('tau_xy = ', tau_xy_grad * grid_ave2, ' kPa')

For the checking block, the effective stresses are:
sigma_x =  8506.09744  kPa
sigma_y =  4939.024320000002  kPa
sigma_z =  8711.89012  kPa
tau_xy =  3089.1759390766183  kPa


In [21]:
grid_top2 = 2503.424
grid_bottom2 = 2596.873
grid_ave2 = (grid_top2 + grid_bottom2)/2

print('For the checking block, the effective stresses are:')
print('sigma_x = ', (sigma_x_grad - 10) * grid_ave2, ' kPa')
print('sigma_y = ', (sigma_y_grad - 10) * grid_ave2, ' kPa')
print('sigma_z = ', (22.7 - 10) * grid_ave2, ' kPa')
print('tau_xy = ', tau_xy_grad * grid_ave2, ' kPa')

For the checking block, the effective stresses are:
sigma_x =  31621.8414  kPa
sigma_y =  18361.06920000001  kPa
sigma_z =  32386.88595  kPa
tau_xy =  11484.165598998463  kPa


## PORO/PERMX pairs are NOT sampled more than once (from file names instead of files)

In [38]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from pathlib import Path
import sys
import re

# Setup for sampling
# Note: 1) stress gradients are effective ones after subtracting 10; 
#       2) stress gradients are negative due to CMG DIR DOWN convention
params = ['E_GPa', 'PR', 'SH_MPa/km', 'Sh_MPa/km', 'Sv_MPa/km', 'SH_azi_deg']
# l_bounds = [15e6, 0.2, 25 * 0.9, 14.6 * 0.9, 22.7 * 0.9, 290]
# u_bounds = [25e6, 0.4, 25 * 1.1, 14.6 * 1.1, 22.7 * 1.1, 310]
l_bounds = [15e6, 0.2, -15 * 1.1, -4.6 * 1.1, -12.7 * 1.1, 290]
u_bounds = [25e6, 0.4, -15 * 0.9, -4.6 * 0.9, -12.7 * 0.9, 310]
num_samples = 90  # Change as needed

# Load PORO and PERMX file names
property_file_names = np.load('property_file_names.npy')

# sort the file names by the number in the name
def extract_number(filename):
    match = re.search(r"(\d+)", filename)
    return int(match.group(1)) if match else float('inf')

poro_file_names = sorted(
    [name for name in property_file_names if "PORO" in name.upper()],
    key=extract_number
)

permx_file_names = sorted(
    [name for name in property_file_names if "PERMX" in name.upper()],
    key=extract_number
)

# check a few things
if not poro_file_names or not permx_file_names:
    print("Error: PORO or PERMX file names not found.")
    sys.exit(1)

if len(poro_file_names) != len(permx_file_names):
    raise ValueError(f"Number of PORO file names ({len(poro_file_names)}) does not match number of PERMX file names ({len(permx_file_names)})")

num_pairs = len(poro_file_names)

if num_samples > num_pairs:
    raise ValueError(f"Cannot sample {num_samples} unique poro/permx pairs: only {num_pairs} available.")

# Latin Hypercube Sampling for parameters
sampler = qmc.LatinHypercube(d=len(params))
sample = sampler.random(n=num_samples)
sample_params = qmc.scale(sample, l_bounds, u_bounds)
df_params = pd.DataFrame(np.round(sample_params, 2), columns=params)

# Store poro/permx pairs
df_params["PORO_file"] = [str(poro_file_names[i]) for i in range(num_samples)]
df_params["PERMX_file"] = [str(permx_file_names[i]) for i in range(num_samples)]

# add prefix to file names
prefix = "data_properties/"
df_params["PORO_file"] = df_params["PORO_file"].apply(lambda x: f"{prefix}{x}")
df_params["PERMX_file"] = df_params["PERMX_file"].apply(lambda x: f"{prefix}{x}")

# Calculate stress state parameters
df_params['beta'] = df_params['SH_azi_deg'] - 90  # Rotate from SH to x-axis
df_params['cos_2beta'] = np.cos(np.radians(2 * df_params['beta']))
df_params['sin_2beta'] = np.sin(np.radians(2 * df_params['beta']))

df_params['sigma_x'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 + \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
df_params['sigma_y'] = (df_params['SH_MPa/km'] + df_params['Sh_MPa/km']) / 2 - \
                       (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['cos_2beta']
df_params['tau_xy'] = (df_params['SH_MPa/km'] - df_params['Sh_MPa/km']) / 2 * df_params['sin_2beta']

# Output
df_params.to_csv("sampled_parameters_and_files.csv", index=False)
df_params


,E_GPa,PR,SH_MPa/km,Sh_MPa/km,Sv_MPa/km,SH_azi_deg,PORO_file,PERMX_file,beta,cos_2beta,sin_2beta,sigma_x,sigma_y,tau_xy
0,23815246.81,0.38,-15.25,-4.89,-11.69,292.13,data_properties/JD_BASECASE_5_PORO.dat,data_properties/JD_BASECASE_5_PERMX.dat,202.13,0.716180,0.697915,-13.779813,-6.360187,-3.615202
1,16819178.15,0.30,-15.55,-4.94,-12.34,308.67,data_properties/JD_BASECASE_6_PORO.dat,data_properties/JD_BASECASE_6_PERMX.dat,218.67,0.219165,0.975688,-11.407671,-9.082329,-5.176024
2,21617787.70,0.38,-15.37,-4.17,-11.47,294.57,data_properties/JD_BASECASE_7_PORO.dat,data_properties/JD_BASECASE_7_PERMX.dat,204.57,0.654213,0.756310,-13.433593,-6.106407,-4.235338
3,15459032.96,0.30,-15.70,-4.57,-13.61,299.84,data_properties/JD_BASECASE_8_PORO.dat,data_properties/JD_BASECASE_8_PERMX.dat,209.84,0.504829,0.863219,-12.944373,-7.325627,-4.803816
4,15209390.12,0.33,-15.17,-4.27,-12.67,296.35,data_properties/JD_BASECASE_9_PORO.dat,data_properties/JD_BASECASE_9_PERMX.dat,206.35,0.605988,0.795473,-13.022637,-6.417363,-4.335330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,18444884.43,0.35,-14.78,-4.23,-13.24,303.49,data_properties/JD_BASECASE_194_PORO.dat,data_properties/JD_BASECASE_194_PERMX.dat,213.49,0.391052,0.920368,-11.567802,-7.442198,-4.854943
86,19796537.93,0.25,-15.50,-4.48,-13.08,308.51,data_properties/JD_BASECASE_197_PORO.dat,data_properties/JD_BASECASE_197_PERMX.dat,218.51,0.224611,0.974449,-11.227606,-8.752394,-5.369211
87,19620202.91,0.32,-13.86,-4.64,-12.09,299.06,data_properties/JD_BASECASE_198_PORO.dat,data_properties/JD_BASECASE_198_PERMX.dat,209.06,0.528142,0.849156,-11.684734,-6.815266,-3.914610
88,23335032.27,0.35,-13.83,-5.06,-11.63,298.11,data_properties/JD_BASECASE_199_PORO.dat,data_properties/JD_BASECASE_199_PERMX.dat,208.11,0.556006,0.831179,-11.883084,-7.006916,-3.644718
